In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
from tsne_torch import TorchTSNE
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
import utils.statnetencoder as sne
import importlib
import imp
import torch.nn.functional as F

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch
sys.path.append('/home/gridsan/ckoevesdi/HBvsBrown/')

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [11]:
import utils.brucenet as bn
brucy = bn.BruceNet(pooling_region_size=1e20, pyramid_params=False, dummy_img = torch.zeros(20,
                                     1,
                                     256,
                                     256)).to(device)

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


In [12]:
ostat_labels = ['mean', 'variance', 'bandpass_variance 0', 'variance 1', 'skew 1', 'kurtosis 1', 'variance 2', 'skew 2', 'kurtosis 2', 'variance 3', 'skew 3', 'kurtosis 3', 'variance 4', 'skew 4', 'kurtosis 4', 'variance 5', 'skew 5', 'kurtosis 5', 'edge_mean 1 0', 'edge_variance 1 0', 'edge_correlation (1, 2) 0', 'edge_stop 1 0', 'edge_mean 1 1', 'edge_variance 1 1', 'edge_correlation (1, 2) 1', 'edge_stop 1 1', 'edge_mean 1 2', 'edge_variance 1 2', 'edge_correlation (1, 2) 2', 'edge_stop 1 2', 'edge_mean 1 3', 'edge_variance 1 3', 'edge_correlation (1, 2) 3', 'edge_stop 1 3', 'edge_correlation 1 (0, 1)', 'edge_correlation 1 (0, 2)', 'edge_correlation 1 (0, 3)', 'edge_correlation 1 (1, 2)', 'edge_correlation 1 (1, 3)', 'edge_correlation 1 (2, 3)', 'edge_mean 2 0', 'edge_variance 2 0', 'edge_correlation (2, 3) 0', 'edge_stop 2 0', 'edge_mean 2 1', 'edge_variance 2 1', 'edge_correlation (2, 3) 1', 'edge_stop 2 1', 'edge_mean 2 2', 'edge_variance 2 2', 'edge_correlation (2, 3) 2', 'edge_stop 2 2', 'edge_mean 2 3', 'edge_variance 2 3', 'edge_correlation (2, 3) 3', 'edge_stop 2 3', 'edge_correlation 2 (0, 1)', 'edge_correlation 2 (0, 2)', 'edge_correlation 2 (0, 3)', 'edge_correlation 2 (1, 2)', 'edge_correlation 2 (1, 3)', 'edge_correlation 2 (2, 3)', 'edge_mean 3 0', 'edge_variance 3 0', 'edge_correlation (3, 4) 0', 'edge_stop 3 0', 'edge_mean 3 1', 'edge_variance 3 1', 'edge_correlation (3, 4) 1', 'edge_stop 3 1', 'edge_mean 3 2', 'edge_variance 3 2', 'edge_correlation (3, 4) 2', 'edge_stop 3 2', 'edge_mean 3 3', 'edge_variance 3 3', 'edge_correlation (3, 4) 3', 'edge_stop 3 3', 'edge_correlation 3 (0, 1)', 'edge_correlation 3 (0, 2)', 'edge_correlation 3 (0, 3)', 'edge_correlation 3 (1, 2)', 'edge_correlation 3 (1, 3)', 'edge_correlation 3 (2, 3)', 'edge_mean 4 0', 'edge_variance 4 0', 'edge_stop 4 0', 'edge_mean 4 1', 'edge_variance 4 1', 'edge_stop 4 1', 'edge_mean 4 2', 'edge_variance 4 2', 'edge_stop 4 2', 'edge_mean 4 3', 'edge_variance 4 3', 'edge_stop 4 3', 'edge_correlation 4 (0, 1)', 'edge_correlation 4 (0, 2)', 'edge_correlation 4 (0, 3)', 'edge_correlation 4 (1, 2)', 'edge_correlation 4 (1, 3)', 'edge_correlation 4 (2, 3)', 'phase_correlation 1 er (0, 1)', 'phase_correlation 1 er (0, 2)', 'phase_correlation 1 er (0, 3)', 'phase_correlation 1 er (1, 2)', 'phase_correlation 1 er (1, 3)', 'phase_correlation 1 er (2, 3)', 'phase_correlation (1, 2) ei*di 0', 'phase_correlation (1, 2) er*di 0', 'phase_correlation (1, 2) ei*di 1', 'phase_correlation (1, 2) er*di 1', 'phase_correlation (1, 2) ei*di 2', 'phase_correlation (1, 2) er*di 2', 'phase_correlation (1, 2) ei*di 3', 'phase_correlation (1, 2) er*di 3', 'phase_correlation 2 er (0, 1)', 'phase_correlation 2 er (0, 2)', 'phase_correlation 2 er (0, 3)', 'phase_correlation 2 er (1, 2)', 'phase_correlation 2 er (1, 3)', 'phase_correlation 2 er (2, 3)', 'phase_correlation (2, 3) ei*di 0', 'phase_correlation (2, 3) er*di 0', 'phase_correlation (2, 3) ei*di 1', 'phase_correlation (2, 3) er*di 1', 'phase_correlation (2, 3) ei*di 2', 'phase_correlation (2, 3) er*di 2', 'phase_correlation (2, 3) ei*di 3', 'phase_correlation (2, 3) er*di 3', 'phase_correlation 3 er (0, 1)', 'phase_correlation 3 er (0, 2)', 'phase_correlation 3 er (0, 3)', 'phase_correlation 3 er (1, 2)', 'phase_correlation 3 er (1, 3)', 'phase_correlation 3 er (2, 3)', 'phase_correlation (3, 4) ei*di 0', 'phase_correlation (3, 4) er*di 0', 'phase_correlation (3, 4) ei*di 1', 'phase_correlation (3, 4) er*di 1', 'phase_correlation (3, 4) ei*di 2', 'phase_correlation (3, 4) er*di 2', 'phase_correlation (3, 4) ei*di 3', 'phase_correlation (3, 4) er*di 3', 'phase_correlation 4 er (0, 1)', 'phase_correlation 4 er (0, 2)', 'phase_correlation 4 er (0, 3)', 'phase_correlation 4 er (1, 2)', 'phase_correlation 4 er (1, 3)', 'phase_correlation 4 er (2, 3)']

In [13]:
print(ostat_labels[89])

edge_stop 4 1


In [14]:
# Path to the image
from PIL import Image

#original image is x
image_path = "/home/gridsan/ckoevesdi/HBvsBrown/chequered_0054.jpg"
x = Image.open(image_path)
x_gray = x.convert("L")
# Convert the PIL Image to a NumPy array
image_array = np.array(x_gray)
to_tensor = transforms.ToTensor()
x_tensor = to_tensor(x_gray)

## HB is y
image_path = "/home/gridsan/ckoevesdi/HBvsBrown/chequered_0054_output.jpg"
y = Image.open(image_path)
print(y)
y = y.convert("L")

# Convert the PIL Image to a NumPy array
image_array = np.array(y)
to_tensor = transforms.ToTensor()
y_tensor = to_tensor(y)

## 50 from brown is z
image_path = "/home/gridsan/ckoevesdi/HBvsBrown/testing_manualstats_HeegerBergen_chequered.png"
z = Image.open(image_path)
z = z.convert("L")

# Convert the PIL Image to a NumPy array
image_array = np.array(z)
to_tensor = transforms.ToTensor()
z_tensor = to_tensor(z)


testkit = torch.stack([x_tensor, y_tensor, z_tensor]).to(device)

<PIL.JpegImagePlugin.JpegImageFile image mode=L size=256x256 at 0x7FCA6C5EF280>


In [22]:
print("Size of the image:", x.size)

Size of the image: (256, 256)


In [15]:
statistics3 = brucy(testkit)  # This should output a tensor of shape [2, 150]
print(statistics3.shape)

torch.Size([3, 152])


In [16]:
variance = [i for i, label in enumerate(ostat_labels) if 'variance' in label]
mean = [i for i, label in enumerate(ostat_labels) if 'mean' in label]
skew = [i for i, label in enumerate(ostat_labels) if 'skew' in label]
kurtosis = [i for i, label in enumerate(ostat_labels) if 'kurtosis' in label]

heegerbergen_indices = variance + mean + skew + kurtosis

In [17]:
print(heegerbergen_indices)

[1, 2, 3, 6, 9, 12, 15, 19, 23, 27, 31, 41, 45, 49, 53, 63, 67, 71, 75, 85, 88, 91, 94, 0, 18, 22, 26, 30, 40, 44, 48, 52, 62, 66, 70, 74, 84, 87, 90, 93, 4, 7, 10, 13, 16, 5, 8, 11, 14, 17]


In [18]:
for j in heegerbergen_indices:
    print(ostat_labels[j])
    print("-" * 20)  # Separator for clarity
    for i in range(3):
        print(statistics3[i, j])
    print("\n") 

variance
--------------------
tensor(0.4251, device='cuda:0')
tensor(0.4259, device='cuda:0')
tensor(0.4277, device='cuda:0')


bandpass_variance 0
--------------------
tensor(0.3751, device='cuda:0')
tensor(0.3764, device='cuda:0')
tensor(0.3751, device='cuda:0')


variance 1
--------------------
tensor(0.3394, device='cuda:0')
tensor(0.3414, device='cuda:0')
tensor(0.3412, device='cuda:0')


variance 2
--------------------
tensor(0.5246, device='cuda:0')
tensor(0.5162, device='cuda:0')
tensor(0.5232, device='cuda:0')


variance 3
--------------------
tensor(0.7192, device='cuda:0')
tensor(0.6910, device='cuda:0')
tensor(0.7183, device='cuda:0')


variance 4
--------------------
tensor(0.9549, device='cuda:0')
tensor(0.8879, device='cuda:0')
tensor(0.9545, device='cuda:0')


variance 5
--------------------
tensor(1.1779, device='cuda:0')
tensor(1.0451, device='cuda:0')
tensor(1.1772, device='cuda:0')


edge_variance 1 0
--------------------
tensor(0.6801, device='cuda:0')
tensor(0.651

In [9]:
all_indices = set(range(len(ostat_labels)))
remaining_indices = sorted(list(all_indices - set(heegerbergen_indices)))
for j in remaining_indices:
    print(ostat_labels[j])
    print("-" * 20)  # Separator for clarity
    for i in range(3):
        print(statistics3[i, j])
    print("\n") 

edge_correlation (1, 2) 0
--------------------
tensor(0.7775, device='cuda:0')
tensor(0.8564, device='cuda:0')
tensor(0.6037, device='cuda:0')


edge_stop 1 0
--------------------
tensor(0.5800, device='cuda:0')
tensor(0.6667, device='cuda:0')
tensor(0.3738, device='cuda:0')


edge_correlation (1, 2) 1
--------------------
tensor(0.2913, device='cuda:0')
tensor(0.4247, device='cuda:0')
tensor(0.2512, device='cuda:0')


edge_stop 1 1
--------------------
tensor(0.0845, device='cuda:0')
tensor(0.1685, device='cuda:0')
tensor(0.0632, device='cuda:0')


edge_correlation (1, 2) 2
--------------------
tensor(0.0741, device='cuda:0')
tensor(0.1758, device='cuda:0')
tensor(0.1897, device='cuda:0')


edge_stop 1 2
--------------------
tensor(0.0767, device='cuda:0')
tensor(0.0862, device='cuda:0')
tensor(0.0769, device='cuda:0')


edge_correlation (1, 2) 3
--------------------
tensor(0.2913, device='cuda:0')
tensor(0.4250, device='cuda:0')
tensor(0.2511, device='cuda:0')


edge_stop 1 3
-------

In [ ]:
# Path to the image
from PIL import Image
image_path = "/home/gridsan/ckoevesdi/Testobreal/43.jpg"

# Read the image
y = Image.open(image_path)
y = y.convert("L")

# Convert the PIL Image to a NumPy array
image_array = np.array(y)
to_tensor = transforms.ToTensor()
y_tensor = to_tensor(y)
yy = torch.stack([y_tensor, y_tensor]).to(device)
print(yy.shape)

In [ ]:
# Path to the image
from PIL import Image
image_path = "/home/gridsan/ckoevesdi/Testobreal/orig.jpg"

# Read the image
y = Image.open(image_path)
y = y.convert("L")

# Convert the PIL Image to a NumPy array
image_array = np.array(y)
to_tensor = transforms.ToTensor()
y_tensor = to_tensor(y)
zz = torch.stack([y_tensor, y_tensor]).to(device)
print(zz.shape)

In [6]:
statistics3 = brucy(zz)  # This should output a tensor of shape [2, 150]
statistics3 = statistics3[0,:]
print(statistics3)

tensor([ 1.7058e+00,  8.0517e-01,  8.1627e-02,  1.1353e+00,  1.1008e+00,
         1.0808e+00,  1.5949e+00,  1.5414e+00,  1.5111e+00,  2.2290e+00,
         2.1419e+00,  2.0943e+00,  3.1081e+00,  2.9692e+00,  2.8972e+00,
         4.3199e+00,  4.1002e+00,  3.9893e+00,  3.6685e-01,  3.4274e-01,
         5.1597e-01,  9.8224e-02,  2.3651e-01,  1.3014e-01,  1.4534e-01,
         3.8183e-02,  2.3796e-01,  1.7185e-01,  2.7811e-01,  5.3021e-02,
         3.9533e-01,  3.4713e-01,  5.9272e-01,  3.9181e-02,  1.6927e-01,
         1.1434e-01,  2.8802e-01,  1.1193e-01,  1.7591e-01,  1.7174e-01,
         7.1330e-01,  7.7737e-01,  9.3105e-01,  3.1284e-01,  3.5216e-01,
         1.8018e-01,  2.2466e-01,  1.2615e-01,  4.9931e-01,  4.3140e-01,
         5.7732e-01,  1.6890e-01,  8.7638e-01,  1.1124e+00,  1.1984e+00,
         4.5787e-01,  2.8173e-01,  2.8274e-01,  7.8357e-01,  2.0124e-01,
         3.0736e-01,  5.0111e-01,  1.1278e+00,  1.1807e+00,  1.3282e+00,
         4.5527e-01,  6.0029e-01,  2.8542e-01,  3.6

In [7]:
statistics = brucy(yy)  # This should output a tensor of shape [2, 150]
statistics = statistics[0,:]

In [8]:
statistics2 = brucy(xx)  # This should output a tensor of shape [2, 150]
statistics2 = statistics2[0,:]

In [ ]:
print(statistics2)

In [ ]:
print(statistics)

In [15]:
import torch
import torch.nn.functional as F


# Slice the vectors to get the first 50 and last 50 elements
yy_first50 = statistics[20:50]
zz_first50 = statistics3[20:50]
yy_last50 = statistics[100:130]
zz_last50 = statistics3[100:130]

# Compute the MSE between the first 50 elements of each vector
mse_first50 = F.mse_loss(yy_first50, zz_first50)

# Compute the MSE between the last 50 elements of each vector
mse_last50 = F.mse_loss(yy_last50, zz_last50)

# Print the results
print(f"MSE for first 50 elements: {mse_first50.item()}")
print(f"MSE for last 50 elements: {mse_last50.item()}")


MSE for first 50 elements: 0.0008842774550430477
MSE for last 50 elements: 0.006435114424675703


In [12]:
import torch
import torch.nn.functional as F


# Slice the vectors to get the first 50 and last 50 elements
yy_first50 = statistics[:50]
zz_first50 = statistics3[:50]
yy_last50 = statistics[50:100]
zz_last50 = statistics3[50:100]

# Compute the MSE between the first 50 elements of each vector
mse_first50 = F.mse_loss(statistics, statistics3)

# Compute the MSE between the last 50 elements of each vector
mse_last50 = F.mse_loss(statistics2, statistics3)

# Print the results
print(f"MSE for first 50 elements: {mse_first50.item()}")
print(f"MSE for last 50 elements: {mse_last50.item()}")


MSE for first 50 elements: 0.0041005113162100315
MSE for last 50 elements: 0.0033451493363827467


In [ ]:
print(yy)